In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import pandas as pd
import seaborn as sns
import os
import pickle
from tqdm import tqdm
import networkx as nx

In [2]:
import heapq

In [3]:
from create_graph.create_graph import generate_distances
NUM_SHOWERS_IN_BRICK = 200

In [4]:
import os
import psutil
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, log_loss
from sklearn.metrics import precision_recall_curve
from IPython.display import clear_output
import sys
sys.path.append("..")
from opera_tools import plot_graphx, DISTANCE, scattering_estimation_loss
from sklearn.linear_model import TheilSenRegressor
from copy import deepcopy      
from collections import Counter

process = psutil.Process(os.getpid())

In [5]:
from functools import total_ordering

@total_ordering
class ClusterHDBSCAN(object):
    def __init__(self, weight: float, cl_size: int, clusters: list = None, nodes: list = None):
        # init
        self.nodes = set()
        self.nodes_in = Counter()
        self.nodes_out = Counter()
        
        self.weights_nodes_dict = defaultdict(set)
        
        self.weight_death = weight
        self.lambda_death = 1. / (weight + 1e-5)
        
        self.weight_birth = weight
        self.lambda_birth = 1. / (weight + 1e-5)
        
        
        self.children = []
        self.falling_out_points = []
        
        assert clusters is not None or nodes is not None
        if clusters is not None:
            for cluster in clusters:
                self.nodes.update(cluster.nodes)
                self.nodes_in.update(cluster.nodes_in)
                self.nodes_out.update(cluster.nodes_out)
                self.weights_nodes_dict[weight].update(cluster.nodes)
                if cluster.is_cluster:
                    cluster.set_weight_birth(weight)
                    self.children.append(cluster)
                else:
                    self.falling_out_points.append(cluster)
        else:
            self.nodes.update(nodes)
            self.nodes_out.update(nodes)
            self.weights_nodes_dict[weight].update(nodes)
        self.frozennodes = frozenset(self.nodes)
        self.__hash = hash(self.frozennodes)
        self.listnodes = list(self.nodes)
        self.npnodes = np.array(list(self.nodes)).astype(np.int32)
        self.cl_size = cl_size
        self.is_cluster = len(self) >= cl_size
        self.is_noise = not self.is_cluster
        self.stability = None
        
        
    def append(self, weight: float, clusters: list):
        """
        Adding
        """
        for cluster in clusters:
            self.nodes.update(cluster.nodes)
            self.weights_nodes_dict[weight].update(cluster.nodes)
        self.weight_birth = weight
        self.lambda_birth = 1 / (weight + 1e-5)
        self.frozennodes = frozenset(self.nodes)
        self.__hash = hash(self.frozennodes)
        self.listnodes = list(self.nodes)
        self.npnodes = np.array(list(self.nodes)).astype(np.int32)
        self.is_cluster = len(self) >= self.cl_size
        self.is_noise = not self.is_cluster
        return self
    
    def __iter__(self):
        for child in self.children:
            yield child
    
    def __contains__(self, node):
        return node in self.nodes
    
    def __len__(self):
        return len(self.nodes)
    
    def __hash__(self):
        return self.__hash
    
    def __eq__(self, other):
        return self.__hash == other.__hash

    def __lt__(self, other):
        return self.__hash < other.__hash
    
    def set_weight_birth(self, weight: float):
        self.weight_birth = weight
        self.lambda_birth = 1 / (weight + 1e-5)
        
    def calculate_stability(self):
        self.stability = 0.
        self.lambda_birth = 1 / (max(self.weights_nodes_dict.keys()) + 1e-5)
        norm = self.lambda_birth
        for weight in self.weights_nodes_dict:
            self.stability += len(self.weights_nodes_dict[weight]) * (1 / (weight + 1e-5) - self.lambda_birth) * norm
            
def calc_stabilities(root):
    root.calculate_stability()
    for child in root:
        calc_stabilities(child)
        
        
def class_disbalance(cluster, graph):
    subgraph = graph.subgraph(cluster.nodes)
    signal = []
    for _, node in subgraph.nodes(data=True):
        signal.append(node['signal'])
    return list(zip(*np.unique(signal, return_counts=True)))

def flat_clusters(root):
    if root.is_cluster:
        yield root
        
    for child in root:
        for cluster in flat_clusters(child):
            yield cluster
            
def reed_stabilities(root, level=0):
    print('    ' * (level - 1) + '+---' * (level > 0), end='')
    print('len={}'.format(len(root)), end=' ')
    print('stability={:.2f}'.format(root.stability))
    for child in root:
        reed_stabilities(child, level + 1)

def print_class_disbalance_for_all_clusters(root, graph, level=0):
    class_disbalance_tuples = class_disbalance(root, graph)

    print('    ' * (level - 1) + '+---' * (level > 0), end='')
    print('len={}'.format(len(root)))
    print('    ' * (level), end='')
    print(class_disbalance_tuples, end=' ')
    print('stability={:.3f}'.format(root.stability))
    for child in root:
        print_class_disbalance_for_all_clusters(child, graph, level + 1)
        
def leaf_clusters(root):
    if root.is_cluster and len(root.children) == 0:
        yield root
        
    for child in root:
        for cluster in leaf_clusters(child):
            yield cluster

def max_level_clusters(root, level=0, max_level=2):
    if level == max_level and root.is_cluster:
        yield root
        
    for child in root:
        for cluster in max_level_clusters(child, level=level+1, max_level=max_level):
            yield cluster
            
            
def recalc_tree(root):
    weights_children = 0
    for child in root:
        weights_children += recalc_tree(child)
    if weights_children > root.stability:
        root.stability = weights_children
    else:
        root.children.clear()
    
    return root.stability


In [6]:
from math import fabs, sqrt, log
def rms_integral_root_closed_py(basetrack_left, basetrack_right):
    EPS = 1e-6
    dz = basetrack_right['features']['SZ'] - basetrack_left['features']['SZ']
    dx = basetrack_left['features']['SX'] - (basetrack_right['features']['SX'] - basetrack_right['features']['TX'] * dz)
    dy = basetrack_left['features']['SY'] - (basetrack_right['features']['SY'] - basetrack_right['features']['TY'] * dz)
    dtx = (basetrack_left['features']['TX'] - basetrack_right['features']['TX'])
    dty = (basetrack_left['features']['TY'] - basetrack_right['features']['TY'])
    
    a = (dtx * dz) ** 2 + (dty * dz) ** 2
    b = 2 * (dtx * dz * dx +  dty * dz * dy)
    c = dx ** 2 + dy ** 2
    if a == 0.:
        return fabs(sqrt(c))
    discriminant = (b ** 2 - 4 * a * c)
    log_denominator = 2 * sqrt(a) * sqrt(a + b + c) + 2 * a + b + EPS
    log_numerator = 2 * sqrt(a) * sqrt(c) + b + EPS
    first_part = ( (2 * a + b) * sqrt(a + b + c) - b * sqrt(c) ) / (4 * a)
    
    if fabs(discriminant) < EPS:
        return fabs(first_part)
    else: 
        result = fabs((discriminant * log(log_numerator / log_denominator) / (8 * sqrt(a * a * a)) + first_part))
        return result


def class_disbalance_graphx(graphx):
    signal = []
    for _, node in graphx.nodes(data=True):
        signal.append(node['signal'])
    return list(zip(*np.unique(signal, return_counts=True)))

def class_disbalance_graphx__(graphx):
    signal = []
    for _, node in graphx.nodes(data=True):
        signal.append(node['signal'])
    return np.unique(signal, return_counts=True)

In [7]:
from math import sqrt
from sklearn.linear_model import TheilSenRegressor, LinearRegression, HuberRegressor

def estimate_params(cluster, angle=0.03):
    x, y, z = estimate_start_xyz(cluster)
    tx, ty = estimate_txty(cluster)
    n = 0
    for i, node in cluster.nodes(data=True):
        #print(x, node['features']['SX'])
        dx = node['features']['SX'] - x
        dy = node['features']['SY'] - y
        dz = node['features']['SZ'] - z 
        dx = dx / dz - tx
        dy = dy / dz - ty
        dz = dz / dz
        if sqrt(dx**2 + dy**2) < angle:
            n += 1
            
    return n / len(cluster), x, y, z, tx, ty


def estimate_start_xyz(cluster, k=3, shift_x=0., shift_y=0., shift_z=-2000.):
    xs = []
    ys = []
    zs = []

    for i, node in cluster.nodes(data=True):
        xs.append(node['features']['SX'])
        ys.append(node['features']['SY'])
        zs.append(node['features']['SZ'])
    
    xs = np.array(xs)
    ys = np.array(ys)
    zs = np.array(zs)
    
    argosorted_z = np.argsort(zs)
    
    x = np.median(np.median(xs[argosorted_z][:k])) + shift_x
    y = np.median(np.median(ys[argosorted_z][:k])) + shift_y
    z = np.median(np.median(zs[argosorted_z][:k])) + shift_z
    
    return x, y, z

def estimate_txty(cluster, k=20):
    xs = []
    ys = []
    zs = []
    tx = []
    ty = []

    for i, node in cluster.nodes(data=True):
        xs.append(node['features']['SX'])
        ys.append(node['features']['SY'])
        zs.append(node['features']['SZ'])
        tx.append(node['features']['TX'])
        ty.append(node['features']['TY'])
        
    
    xs = np.array(xs)
    ys = np.array(ys)
    zs = np.array(zs)
    tx = np.array(tx)
    ty = np.array(ty)
    
    
    argosorted_z = np.argsort(zs)
    lr = TheilSenRegressor()
    lr.fit(zs[argosorted_z][:k].reshape((-1, 1)), xs[argosorted_z][:k])
    TX = lr.coef_[0]

    lr.fit(zs[argosorted_z][:k].reshape((-1, 1)), ys[argosorted_z][:k])
    TY = lr.coef_[0]
    
    return TX, TY
    return np.median(np.median(tx[argosorted_z][:k])), np.median(np.median(ty[argosorted_z][:k]))

## Load data

In [8]:
from opera_tools import combine_mc_bg, gen_graphx, gen_x_y_dataset, load_bg, load_mc
pmc = load_mc(filename='mcdata_taue2.root', step=1)
def pmc_to_ship_format(pmc):
    showers = []
    scale = 10000
    for idx in pmc.index:
        shower = pmc.loc[idx]
        
        showers.append(
            {
                'TX': shower['BT_X'] / scale,
                'TY': shower['BT_Y'] / scale,
                'TZ': shower['BT_Z'] / scale,
                'PX': shower['BT_SX'],
                'PY': shower['BT_SY'],
                'PZ': np.ones_like(shower['BT_X']),
                'ele_P': shower['ele_P'],
                'ele_TX': shower['ele_x'] / scale,
                'ele_TY': shower['ele_y'] / scale,
                'ele_TZ': shower['ele_z']  / scale,
                'ele_PX': shower['ele_sx'],
                'ele_PY': shower['ele_sy'],
                'ele_PZ': 1.
            }
        )
    return showers
selected_showers = pmc_to_ship_format(pmc)


numtracks reduction by cuts:  [18724, 18679, 9616, 9106]
len(pmc): 9106


In [9]:
selected_showers = [selected_shower for selected_shower in selected_showers if len(selected_shower['PX']) > 70]

In [10]:
selected_showers = [selected_shower for selected_shower in selected_showers if len(selected_shower['PX']) < 3000]

In [11]:
len(selected_showers)

8019

In [12]:
bricks = []
NUM_SHOWERS_IN_BRICK = 200

In [13]:
%%time
scale = 10000
bricks = []
for i in range(len(selected_showers) // NUM_SHOWERS_IN_BRICK):
    node_id = 0
    graphx = nx.DiGraph()
    nodes_to_add = []
    showers_data = []
    for j in range(NUM_SHOWERS_IN_BRICK):
        selected_shower = selected_showers[i * NUM_SHOWERS_IN_BRICK + j]
        showers_data.append(
            {
            'numtracks': len(selected_shower['PX']),
            'signal': j,
            'ele_P': selected_shower['ele_P'],
            'ele_SX': selected_shower['ele_TX'] * scale,
            'ele_SY': selected_shower['ele_TY'] * scale,
            'ele_SZ': selected_shower['ele_TZ'] * scale,
            'ele_TX': selected_shower['ele_PX'] / selected_shower['ele_PZ'],
            'ele_TY': selected_shower['ele_PY'] / selected_shower['ele_PZ']
            }
        )
        for k in range(len(selected_shower['PX'])):
            nodes_to_add.append(
                (
                    node_id,
                    {
                        'features': {
                            'SX': selected_shower['TX'][k] * scale,
                            'SY': selected_shower['TY'][k] * scale,
                            'SZ': selected_shower['TZ'][k] * scale,
                            'TX': selected_shower['PX'][k] / selected_shower['PZ'][k],
                            'TY': selected_shower['PY'][k] / selected_shower['PZ'][k],
                        },
                        'signal': j
                    }
                )
            )
            node_id += 1
    graphx.add_nodes_from(nodes_to_add)
    graphx.graph['showers_data'] = showers_data
    bricks.append(graphx)

CPU times: user 1min 16s, sys: 2.89 s, total: 1min 19s
Wall time: 1min 19s


In [14]:
len(bricks)

40

In [15]:
bricks[0]

## Функция подсчёта метрики

In [16]:
import operator
from collections import Counter

def run_hdbscan(G, cl_size=20, order=True):
    ####### core_d was deleted => could be returned. Laverage robustness / cluster shaprness.
    edges = []
    for node_id_left, node_id_right, edge in G.edges(data=True):
        node_left = G.node[node_id_left]
        node_right = G.node[node_id_right]
        edges.append(
            (
            node_id_left, node_id_right, edge['weight'], np.sign(node_left['features']['SZ'] - node_right['features']['SZ'])
            )
        )

    #nx.mini
    #edges = list(G.edges(data=True))
    #edges = [(edge[0], # 0
    #          edge[1], # 1
    #          edge[2]['weight'], # 2
    #         ) for edge in edges]
    
    edges = sorted(edges, key = operator.itemgetter(2))

    ### Minimum spanning tree was also thrown
    ### following algo reminds of Kruskal algo but with some modifications

    ### TODO: sort on some mix(i.e. linear) of Z_coord and integrated distance
    
    # init
    clusters = {}
    for node_id in G.nodes():
        clusters[node_id]=ClusterHDBSCAN(cl_size=cl_size, weight=np.inf, nodes=[node_id])


    for i, j, weight, *_ in edges:
        cluster_out = clusters[i]
        cluster_in = clusters[j]

        ### tunable parameter
        #if cluster_in.nodes_in[j] > 1:
        #    continue

        if cluster_in is cluster_out:
            continue

        if cluster_in.is_cluster and cluster_out.is_cluster:
            cluster = ClusterHDBSCAN(weight=weight , cl_size=cl_size, clusters=[cluster_in, cluster_out])
        elif cluster_in.is_cluster and not cluster_out.is_cluster:
            cluster = cluster_in.append(weight=weight, clusters=[cluster_out])
        elif cluster_out.is_cluster and not cluster_in.is_cluster:
            cluster = cluster_out.append(weight=weight, clusters=[cluster_in])
        else:
            cluster = ClusterHDBSCAN(weight=weight, cl_size=cl_size, clusters=[cluster_in, cluster_out])

        cluster.nodes_out[i] += 1
        cluster.nodes_in[j] += 1

        clusters.update({l: cluster for l in cluster.nodes})
        
    clusters = list(set(clusters.values()))

    ### choose biggest cluster
    root = clusters[0]
    length = len(clusters[0])
    for cluster in clusters:
        if len(cluster) > length:
            length = len(cluster)
            root = cluster
    
    calc_stabilities(root)
    #recalc_tree(root)
    clusters = list(leaf_clusters(root))
    return clusters, root

In [17]:
def run_hdbscan_on_brick(graphx, min_cl=40, cl_size=40, order=True):
    connected_components = []
    for cnn in nx.connected_components(nx.Graph(graphx)):
        if len(cnn) > min_cl:
            connected_components.append(nx.DiGraph(graphx.subgraph(cnn)))
    clusters = []
    roots = []
    for G in connected_components:
        if len(G) < 100:
            clusters.append(G)
        else:
            clusters_hdbscan, root_hdbscan = run_hdbscan(G, cl_size=cl_size, order=order)
            roots.append(root_hdbscan)
            clusters.extend(clusters_hdbscan)
    
    return graphx, clusters, roots

In [18]:
def estimate_e(cluster, angle=0.005):
    x, y, z = estimate_start_xyz(cluster)
    tx, ty = estimate_txty(cluster)
    n = 0
    for i, node in cluster.nodes(data=True):
        dx = node['features']['SX'] - x
        dy = node['features']['SY'] - y
        dz = node['features']['SZ'] - z 
        dx = dx / dz - tx
        dy = dy / dz - ty
        dz = dz / dz
        if sqrt(dx**2 + dy**2) < angle:
            n += 1
            
    return n / len(cluster)

In [19]:
def return_train_data(root, graphx):
    for child in root:
        for Xy in return_train_data(child, graphx):
            yield Xy
    if root.is_cluster and root.children:
        yield get_train_data(root, graphx)
        
def get_cluster_params(root, graphx):
    graphx_root = graphx.subgraph(root.nodes)
    _, x, y, z, tx, ty = estimate_params(graphx_root)
    return x, y, z, tx, ty


def all_clusters_params(root, graphx):
    for child in root:
        for x, y, z, tx, ty in all_clusters_params(child, graphx):
            yield x, y, z, tx, ty
    if root.is_cluster:
        yield get_cluster_params(root, graphx)
        
        
def estimate_params(cluster):
    x, y, z = estimate_start_xyz(cluster)
    tx, ty = estimate_txty(cluster)
    return x, y, z, tx, ty

def get_train_data(root, graphx):
    graphx_root = graphx.subgraph(root.nodes)
    children = [graphx.subgraph(root.children[0].nodes), 
                graphx.subgraph(root.children[1].nodes)]
    x_r, y_r, z_r, tx_r, ty_r = estimate_params(graphx_root)
    x_c1, y_c1, z_c1, tx_c1, ty_c1 = estimate_params(children[0])
    x_c2, y_c2, z_c2, tx_c2, ty_c2 = estimate_params(children[1])
    X = [
            estimate_e(graphx_root, angle=0.01), 
            estimate_e(graphx_root, angle=0.03), 
            estimate_e(graphx_root, angle=0.05),
            estimate_e(children[0], angle=0.01),
            estimate_e(children[0], angle=0.03),
            estimate_e(children[1], angle=0.01),
            estimate_e(children[1], angle=0.03),
            sqrt((tx_r - tx_c1)**2 + (ty_r - ty_c1)**2),
            sqrt((tx_r - tx_c2)**2 + (ty_r - ty_c2)**2),
            root.lambda_birth, 
            root.stability, 
            root.children[0].stability, 
            root.children[1].stability
        ]
    cl_dis = np.array(list(zip(*class_disbalance_graphx(graphx_root)))[1])
    if len(cl_dis) == 1:
        y = 1
    elif (cl_dis / cl_dis.sum()).max() > 0.9 and cl_dis.min() < 40:
        y = 1
    else:
        y = 0
    return X, y

def return_train_data(root, graphx):
    for child in root:
        for Xy in return_train_data(child, graphx):
            yield Xy
    if root.is_cluster and root.children:
        Xy = get_train_data(root, graphx)
        root.Xy = Xy
        yield Xy

In [36]:
def run_gen_graphx(graphx, layers=2, threshold=250):
    graphx_nodes = list(graphx.nodes(data=True))
    edges = list(graphx.edges())
    graphx.remove_edges_from(edges)
    ebunch = generate_distances(graphx_nodes, layers=layers, threshold=threshold)
    graphx.add_edges_from(ebunch)
    return graphx, ebunch

---------

In [37]:
graphx, ebunch = run_gen_graphx(bricks[1], layers=5, threshold=400)


  3%|▎         | 2750/108505 [00:15<12:07, 145.31it/s]


  5%|▌         | 5512/108505 [00:31<10:34, 162.43it/s]


  7%|▋         | 7957/108505 [00:47<12:17, 136.28it/s]


 10%|▉         | 10564/108505 [01:02<09:33, 170.81it/s]


 12%|█▏        | 12663/108505 [01:23<12:14, 130.49it/s]


...

 93%|█████████▎| 101253/108505 [10:23<00:50, 143.54it/s]


 95%|█████████▌| 103392/108505 [10:37<00:30, 168.40it/s]


 98%|█████████▊| 105946/108505 [10:52<00:13, 186.00it/s]


100%|█████████▉| 108316/108505 [11:07<00:01, 137.49it/s]


100%|██████████| 108505/108505 [11:08<00:00, 162.30it/s]


In [68]:
graphx_test, ebunch_test = run_gen_graphx(bricks[2], layers=5, threshold=400)


  3%|▎         | 2831/104220 [00:15<07:19, 230.76it/s]


  5%|▌         | 5274/104220 [00:31<11:48, 139.65it/s]


  8%|▊         | 7826/104220 [00:46<11:22, 141.30it/s]


 10%|█         | 10485/104220 [01:02<10:47, 144.66it/s]


 12%|█▏        | 12954/104220 [01:17<11:01, 138.03it/s]


...

 91%|█████████ | 94850/104220 [09:41<01:02, 149.92it/s]


 93%|█████████▎| 97249/104220 [09:57<00:42, 163.93it/s]


 96%|█████████▌| 99633/104220 [10:18<01:06, 69.44it/s]


 98%|█████████▊| 102147/104220 [10:33<00:15, 136.50it/s]


100%|██████████| 104220/104220 [10:46<00:00, 161.22it/s]


In [48]:
graphx.nodes(data = True)[0]

{'features': {'SX': -43371.18625640869,
  'SY': -34427.40201950073,
  'SZ': 1130.0771683454514,
  'TX': 0.085678026,
  'TY': 0.05249354},
 'signal': 0}

In [66]:
dist = [ebunch[i][2]['weight'] for i in range(len(ebunch))]

In [69]:
dist_test = [ebunch_test[i][2]['weight'] for i in range(len(ebunch_test))]

In [71]:
len(dist_test)

4096292

In [72]:
len(dist)

4806582

In [ ]:
#TODO 
#return 'r' ---- v
#generate 2nd graph for test
#train classifier 

In [28]:
#with open('graph_0.pickle', 'wb') as f:
#    pickle.dump(graphx, f, protocol=pickle.HIGHEST_PROTOCOL)

### Classifier

In [119]:
shower_data = torch.FloatTensor(pd.DataFrame(graphx.graph['showers_data']).values)

In [121]:
shower_data.shape #numtracks	signal	ele_P	ele_SX	ele_SY	ele_SZ	ele_TX	ele_TY

torch.Size([200, 8])

In [98]:
x_ = pd.DataFrame([graphx.nodes(data = True)[i]['features'] for i in range(len(graphx.nodes))])

In [100]:
x = torch.FloatTensor(x_.values) 

In [125]:
x.shape

torch.Size([103926, 5])

In [111]:
edge_attr = torch.log(torch.FloatTensor(dist).view(-1, 1))

In [112]:
edge_attr

tensor([[-0.0267],
        [ 1.9675],
        [ 3.3002],
        ...,
        [ 5.5773],
        [ 5.8146],
        [ 5.1639]])

In [129]:
y = torch.LongTensor([graphx.nodes(data = True)[i]['signal'] for i in range(len(graphx.nodes))])

In [130]:
y.shape

torch.Size([103926])

In [ ]:
dist = [ebunch[i][2]['weight'] for i in range(len(ebunch))]

In [135]:
edges_from = [ebunch[i][0] for i in range(len(ebunch))]
edge_to = [ebunch[i][1] for i in range(len(ebunch))]

In [136]:
edges = np.vstack([edges_from, edge_to])
        
edge_index = torch.LongTensor(edges)

In [137]:
edge_index.shape

torch.Size([2, 4806582])

In [138]:
len(dist)

4806582

In [150]:
from torch_geometric.data import Data

shower = Data(
        x=x,
        edge_index=edge_index,
        shower_data=shower_data,
        pos=x,
        edge_attr=edge_attr,
        y=y
)

In [155]:
shower.x

tensor([[-4.3371e+04, -3.4427e+04,  1.1301e+03,  8.5678e-02,  5.2494e-02],
        [-4.3393e+04, -3.4342e+04,  1.1312e+03,  6.1419e-02,  8.9527e-02],
        [-4.3262e+04, -3.4362e+04,  2.4023e+03,  8.1178e-02,  5.1911e-02],
        ...,
        [ 6.0155e+04,  1.6126e+03,  3.4225e+04,  3.9822e-02, -5.1669e-01],
        [ 6.1945e+04, -9.7280e+02,  3.4225e+04, -1.4242e-01,  3.2809e-03],
        [ 6.2139e+04, -1.4835e+03,  3.4227e+04, -2.0823e-02, -3.7811e-01]])

In [172]:
showers = [shower]

In [173]:
output_file='./train_.pt'

torch.save(showers, output_file)

---------

In [ ]:
import itertools
from operator import itemgetter
def k_nearest_cut_succ(graphx, k):
    for node_id in tqdm(graphx.nodes()):
        successors = list(graphx.successors(node_id))
        if len(successors) <= k:
            continue
        edges = list(itertools.product([node_id], successors))
        weights = []
        for edge in edges:
            weights.append(graphx[edge[0]][edge[1]]['weight'])
        weights, edges = [list(x) for x in zip(*sorted(zip(weights, edges), key=itemgetter(0)))]
        edges_to_remove = edges[k:]
        if len(edges_to_remove):
            graphx.remove_edges_from(edges_to_remove)
    return graphx

def k_nearest_cut_pred(graphx, k):
    for node_id in tqdm(graphx.nodes()):
        predecessors = list(graphx.predecessors(node_id))
        if len(predecessors) <= k:
            continue
        edges = list(itertools.product(predecessors, [node_id]))
        weights = []
        for edge in edges:
            weights.append(graphx[edge[0]][edge[1]]['weight'])
        weights, edges = [list(x) for x in zip(*sorted(zip(weights, edges), key=itemgetter(0)))]

        edges_to_remove = edges[k:]
        if len(edges_to_remove):
            graphx.remove_edges_from(edges_to_remove)
    return graphx

In [ ]:
from IPython.display import clear_output
X = []
y = []
group = []
clusterized_bricks = []
g = 0
for graphx in tqdm(bricks[:len(bricks) // 4]):
    print(len(X))
    run_gen_graphx(graphx, layers=5, threshold=400)
    k_nearest_cut_succ(graphx, 25)
    k_nearest_cut_pred(graphx, 25)
    graphx, clusters, roots = run_hdbscan_on_brick(graphx=graphx, min_cl=40, cl_size=40)
    for root in roots:
        try:
            X_tmp, y_tmp = list(zip(*list(return_train_data(root, graphx.subgraph(root.nodes)))))
            X.append(np.array(X_tmp))
            y.append(np.array(y_tmp))
            group.append(np.full_like(a=y_tmp, fill_value=g))
            g += 1
        except:
            pass
    clear_output()

-------

In [ ]:
X = np.concatenate(X)
X = np.c_[X, X[:, -1] + X[:, -2] - X[:, -3]]
y = np.concatenate(y)
group = np.concatenate(group)

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
svc = svm.SVC(C=100, gamma=0.1, probability=True)
clf = GridSearchCV(svc, {}, scoring='roc_auc')
clf.fit(X, y, group)

In [ ]:
clf.grid_scores_

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, average_precision_score

plt.figure(figsize=(12, 12))
predictions = clf.predict_proba(X)[:, 1]
plt.hist(predictions[y==1], label='Join cluster', alpha=0.7)
plt.hist(predictions[y==0], label='Divide cluster', alpha=0.7)
plt.legend()
plt.show()


fpr, tpr, _ = roc_curve(y, predictions)
roc_auc = auc(fpr, tpr)


plt.figure(figsize=(12, 12))
lw = 5
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import operator
from collections import Counter
def make_it_trained(root, graphx):
    print('done')
    for child in root:
        make_it_trained(child, graphx)
    if root.is_cluster and root.children:
        Xy = get_train_data(root, graphx)
        X = Xy[0]
        X.append(X[-1] + X[-2] - X[-3])
        root.X = X
        
def run_hdbscan_clf(G, clf, cl_size=20, order=True):
    ####### core_d was deleted => could be returned. Laverage robustness / cluster shaprness.
    edges = []
    for node_id_left, node_id_right, edge in G.edges(data=True):
        node_left = G.node[node_id_left]
        node_right = G.node[node_id_right]
        edges.append(
            (
            node_id_left, node_id_right, edge['weight'], np.sign(node_left['features']['SZ'] - node_right['features']['SZ'])
            )
        )

    #nx.mini
    #edges = list(G.edges(data=True))
    #edges = [(edge[0], # 0
    #          edge[1], # 1
    #          edge[2]['weight'], # 2
    #         ) for edge in edges]
    
    edges = sorted(edges, key = operator.itemgetter(2))

    ### Minimum spanning tree was also thrown
    ### following algo reminds of Kruskal algo but with some modifications

    ### TODO: sort on some mix(i.e. linear) of Z_coord and integrated distance
    
    # init
    clusters = {}
    for node_id in G.nodes():
        clusters[node_id]=ClusterHDBSCAN(cl_size=cl_size, weight=np.inf, nodes=[node_id])


    for i, j, weight, *_ in edges:
        cluster_out = clusters[i]
        cluster_in = clusters[j]

        ### tunable parameter
        #if cluster_in.nodes_in[j] > 1:
        #    continue

        if cluster_in is cluster_out:
            continue

        if cluster_in.is_cluster and cluster_out.is_cluster:
            cluster = ClusterHDBSCAN(weight=weight , cl_size=cl_size, clusters=[cluster_in, cluster_out])
        elif cluster_in.is_cluster and not cluster_out.is_cluster:
            cluster = cluster_in.append(weight=weight, clusters=[cluster_out])
        elif cluster_out.is_cluster and not cluster_in.is_cluster:
            cluster = cluster_out.append(weight=weight, clusters=[cluster_in])
        else:
            cluster = ClusterHDBSCAN(weight=weight, cl_size=cl_size, clusters=[cluster_in, cluster_out])

        cluster.nodes_out[i] += 1
        cluster.nodes_in[j] += 1

        clusters.update({l: cluster for l in cluster.nodes})
        
    clusters = list(set(clusters.values()))

    ### choose biggest cluster
    root = clusters[0]
    length = len(clusters[0])
    for cluster in clusters:
        if len(cluster) > length:
            length = len(cluster)
            root = cluster
    
    calc_stabilities(root)
    make_it_trained(root, G.subgraph(root.nodes))
    recalc_tree_clf(root, clf)
    clusters = list(leaf_clusters(root))
    return clusters, root

TH = 0.8

def recalc_tree_clf(root, clf):
    weights_children = 0
    #try:
    if root.children and root.is_cluster:
        p = clf.predict_proba([root.X])[0][1]
    else:
        p = 1.
    for child in root:
        recalc_tree_clf(child, clf)
    if p > TH:
        root.children.clear()
    return root

def run_hdbscan_on_brick_clf(graphx, clf, min_cl=40, cl_size=40, order=True):
    connected_components = []
    for cnn in nx.connected_components(nx.Graph(graphx)):
        if len(cnn) > min_cl:
            connected_components.append(nx.DiGraph(graphx.subgraph(cnn)))
    clusters = []
    roots = []
    for G in connected_components:
        if len(G) < 100:
            clusters.append(G)
        else:
            clusters_hdbscan, root_hdbscan = run_hdbscan_clf(G, clf, cl_size=cl_size, order=order)
            roots.append(root_hdbscan)
            clusters.extend(clusters_hdbscan)
    clusters_graphx = []
    for cluster in clusters:
        clusters_graphx.append(
            nx.DiGraph(graphx.subgraph(cluster.nodes))
        )
    return graphx, clusters, roots, clusters_graphx

In [ ]:
TH = 0.7

from IPython.display import clear_output
clusterized_bricks = []
for graphx in tqdm(bricks[len(bricks) // 4:]):
    run_gen_graphx(graphx, layers=5, threshold=400)
    k_nearest_cut_succ(graphx, 25)
    k_nearest_cut_pred(graphx, 25)
    graphx, clusters, roots, clusters_graphx = run_hdbscan_on_brick_clf(graphx=graphx, clf=clf, min_cl=40, cl_size=40)
    clusterized_bricks.append(
        {
            'graphx': graphx,
            'clusters': clusters_graphx,
        }
    )
    clear_output()

In [ ]:
for k in range(len(clusterized_bricks[0]['clusters'])):
    print(len(clusterized_bricks[0]['clusters'][k]))
    print(class_disbalance_graphx(clusterized_bricks[0]['clusters'][k]))

In [ ]:
len(clusterized_bricks[0]['clusters'])

In [ ]:
for k in range(len(clusterized_bricks[0]['clusters'])):
    print(len(clusterized_bricks[0]['clusters'][k]))
    print(class_disbalance_graphx(clusterized_bricks[0]['clusters'][k]))

In [ ]:
for k in range(len(clusterized_bricks[0]['clusters'])):
    print(len(clusterized_bricks[0]['clusters'][k]))
    print(class_disbalance_graphx(clusterized_bricks[0]['clusters'][k]))

In [ ]:
6 500
for k in range(len(clusterized_bricks[0]['clusters'])):
    print(len(clusterized_bricks[0]['clusters'][k]))
    print(class_disbalance_graphx(clusterized_bricks[0]['clusters'][k]))

In [ ]:
for k in range(len(clusterized_bricks[0]['clusters'])):
    print(len(clusterized_bricks[0]['clusters'][k]))
    print(class_disbalance_graphx(clusterized_bricks[0]['clusters'][k]))

In [ ]:
plot_graphx((clusterized_bricks[0]['clusters'][9]))
plot_graphx((clusterized_bricks[0]['clusters'][10]))

In [ ]:
NUM_SHOWERS_IN_BRICK

In [ ]:
len(clusterized_bricks)

In [ ]:
with open('../data/opera/clusterized_bricks_{}.pickle'.format(NUM_SHOWERS_IN_BRICK), 'wb') as f:
    pickle.dump(clusterized_bricks, f, protocol=pickle.HIGHEST_PROTOCOL)